In [1]:
from datetime import datetime
import math
import time
import tensorflow as tf

C:\Soft\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def conv_op(input_op, name, kh, kw, n_out, dh, dw, p):
    n_in = input_op.get_shape()[-1].value

    with tf.name_scope(name) as scope:
        kernel = tf.get_variable(
            scope + "w",
            shape=[kh, kw, n_in, n_out],
            dtype=tf.float32,
            initializer=tf.contrib.layers.xavier_initializer_conv2d())
        conv = tf.nn.conv2d(input_op, kernel, (1, dh, dw, 1), padding='SAME')
        bias_init_val = tf.constant(0.0, shape=[n_out], dtype=tf.float32)
        biases = tf.Variable(bias_init_val, trainable=True, name='b')
        z = tf.nn.bias_add(conv, biases)
        activation = tf.nn.relu(z, name=scope)
        p += [kernel, biases]
        return activation

In [3]:
def fc_op(input_op, name, n_out, p):
    n_in = input_op.get_shape()[-1].value

    with tf.name_scope(name) as scope:
        kernel = tf.get_variable(
            scope + "w",
            shape=[n_in, n_out],
            dtype=tf.float32,
            initializer=tf.contrib.layers.xavier_initializer())
        biases = tf.Variable(
            tf.constant(0.1, shape=[n_out], dtype=tf.float32), name='b')
        activation = tf.nn.relu_layer(input_op, kernel, biases, name=scope)
        p += [kernel, biases]
        return activation

In [4]:
def mpool_op(input_op, name, kh, kw, dh, dw):
    return tf.nn.max_pool(
        input_op,
        ksize=[1, kh, kw, 1],
        strides=[1, dh, dw, 1],
        padding='SAME',
        name=name)

In [5]:
def inference_op(input_op, keep_prop):
    p = []

    conv1_1 = conv_op(
        input_op, name="conv1_1", kh=3, kw=3, n_out=64, dh=1, dw=1, p=p)

    conv1_2 = conv_op(
        conv1_1, name="conv1_2", kh=3, kw=3, n_out=64, dh=1, dw=1, p=p)

    pool1 = mpool_op(conv1_2, name="pool1", kh=2, kw=2, dh=2, dw=2)

    conv2_1 = conv_op(
        pool1, name="conv2_1", kh=3, kw=3, n_out=128, dh=1, dw=1, p=p)

    conv2_2 = conv_op(
        conv2_1, name="conv2_2", kh=3, kw=3, n_out=128, dh=1, dw=1, p=p)

    pool2 = mpool_op(conv2_2, name="pool2", kh=2, kw=2, dh=2, dw=2)

    conv3_1 = conv_op(
        pool2, name="conv3_1", kh=3, kw=3, n_out=256, dh=1, dw=1, p=p)

    conv3_2 = conv_op(
        conv3_1, name="conv3_2", kh=3, kw=3, n_out=256, dh=1, dw=1, p=p)

    conv3_3 = conv_op(
        conv3_2, name="conv3_3", kh=3, kw=3, n_out=256, dh=1, dw=1, p=p)

    pool3 = mpool_op(conv3_3, name="pool3", kh=2, kw=2, dh=2, dw=2)

    conv4_1 = conv_op(
        pool3, name="conv4_1", kh=3, kw=3, n_out=512, dh=1, dw=1, p=p)

    conv4_2 = conv_op(
        conv4_1, name="conv4_2", kh=3, kw=3, n_out=512, dh=1, dw=1, p=p)

    conv4_3 = conv_op(
        conv4_2, name="conv4_3", kh=3, kw=3, n_out=512, dh=1, dw=1, p=p)

    pool4 = mpool_op(conv4_3, name="pool4", kh=2, kw=2, dh=2, dw=2)

    conv5_1 = conv_op(
        pool4, name="conv5_1", kh=3, kw=3, n_out=512, dh=1, dw=1, p=p)

    conv5_2 = conv_op(
        conv5_1, name="conv5_2", kh=3, kw=3, n_out=512, dh=1, dw=1, p=p)

    conv5_3 = conv_op(
        conv5_2, name="conv5_3", kh=3, kw=3, n_out=512, dh=1, dw=1, p=p)

    pool5 = mpool_op(conv5_3, name="pool5", kh=2, kw=2, dh=2, dw=2)

    shp = pool5.get_shape()
    flattened_shape = shp[1].value * shp[2].value * shp[3].value
    resh1 = tf.reshape(pool5, [-1, flattened_shape], name="resh1")

    fc6 = fc_op(resh1, name="fc6", n_out=4096, p=p)
    fc6_drop = tf.nn.dropout(fc6, keep_prop, name="fc6_drop")

    fc7 = fc_op(fc6_drop, name="fc7", n_out=4096, p=p)
    fc7_drop = tf.nn.dropout(fc7, keep_prop, name="fc7_drop")

    fc8 = fc_op(fc7_drop, name="fc8", n_out=1000, p=p)
    softmax = tf.nn.softmax(fc8)
    predictions = tf.argmax(softmax, 1)
    return predictions, softmax, fc8, p

In [6]:
def time_tensorflow_run(session, target, feed, info_string):
    num_steps_burn_in = 10
    total_duration = 0.0
    total_duration_squared = 0.0
    for i in range(num_batches + num_steps_burn_in):
        start_time = time.time()
        _ = session.run(target, feed_dict=feed)
        duration = time.time() - start_time
        if i >= num_steps_burn_in:
            if not i % 10:
                print('%s: step %d,duration = %.3f' %
                      (datetime.now(), i - num_steps_burn_in, duration))
            total_duration += duration
            total_duration_squared += duration * duration
    mn = total_duration / num_batches
    vr = total_duration_squared / num_batches - mn * mn
    sd = math.sqrt(vr)
    print('%s: %s across %d steps, %.3f +/- %.3f sec / batch' %
          (datetime.now(), info_string, num_batches, mn, sd))

In [7]:
def run_benchmark():
    with tf.Graph().as_default():
        image_size = 32
        images = tf.Variable(
            tf.random_normal([batch_size, image_size, image_size, 3],
                             dtype=tf.float32,
                             stddev=1e-1))

        keep_prop = tf.placeholder(tf.float32)
        predictions, softmax, fc8, p = inference_op(images, keep_prop)

        init = tf.global_variables_initializer()
        sess = tf.Session()
        sess.run(init)

        time_tensorflow_run(sess, predictions, {keep_prop: 1.0}, "Forward")
        objective = tf.nn.l2_loss(fc8)
        grad = tf.gradients(objective, p)
        time_tensorflow_run(sess, grad, {keep_prop: 0.5}, "Forward-backward")

In [8]:
batch_size = 32
num_batches = 100
run_benchmark()

2019-03-14 11:19:04.829757: step 0,duration = 0.203
2019-03-14 11:19:09.352198: step 10,duration = 0.686
2019-03-14 11:19:16.480106: step 20,duration = 0.655
2019-03-14 11:19:21.777225: step 30,duration = 0.474
2019-03-14 11:19:26.011677: step 40,duration = 0.377
2019-03-14 11:19:29.686850: step 50,duration = 0.353
2019-03-14 11:19:32.927123: step 60,duration = 0.312
2019-03-14 11:19:35.110741: step 70,duration = 0.193
2019-03-14 11:19:37.326861: step 80,duration = 0.516
2019-03-14 11:19:43.251823: step 90,duration = 0.597
2019-03-14 11:19:48.589159: Forward across 100 steps, 0.439 +/- 0.176 sec / batch
2019-03-14 11:20:11.058851: step 0,duration = 1.666
2019-03-14 11:20:26.973275: step 10,duration = 1.345
2019-03-14 11:20:41.882218: step 20,duration = 1.273
2019-03-14 11:20:55.742655: step 30,duration = 1.322
2019-03-14 11:21:10.423343: step 40,duration = 1.285
2019-03-14 11:21:27.528488: step 50,duration = 1.438
2019-03-14 11:21:56.648442: step 60,duration = 2.297
2019-03-14 11:22:12